# © Artur Czarnecki. All rights reserved.
# Intergrax framework – proprietary and confidential.
# Use, modification, or distribution without written permission is prohibited.

# 07 — User Profile Instructions Baseline

## Goal

Introduce the first production-minimal user profile memory flow:

**UserProfile → (profile.system_instructions) → runtime SYSTEM instructions**

This notebook verifies that **user-level system instructions** can be resolved from the stored `UserProfile` and injected as the **first** `system` message in the runtime prompt.

## Scope (baseline)

- No RAG
- No tools
- No web search
- No CoT / reasoning policies beyond default `DIRECT`
- Strict typed fields (no `getattr`)
- One cell at a time, production-ready wiring

## What we will validate

1. A `UserProfile` is persisted in the `UserProfileStore`.
2. `SessionManager.get_user_profile_instructions_for_session()` resolves the user instructions string.
3. `DropInKnowledgeRuntime` injects the final instructions as the first `system` message in `messages_for_llm`.


In [ ]:
import sys, os

from intergrax.llm_adapters.llm_provider_registry import LLMAdapterRegistry
from intergrax.runtime.drop_in_knowledge_mode.engine.runtime_context import RuntimeContext
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

# ==========================================================
# RUNTIME CONFIGURATION (BASELINE: USER PROFILE INSTRUCTIONS)
# ==========================================================

from intergrax.llm_adapters.llm_provider import LLMProvider
from intergrax.runtime.drop_in_knowledge_mode.config import RuntimeConfig
from intergrax.runtime.drop_in_knowledge_mode.engine.runtime import RuntimeEngine

from intergrax.runtime.drop_in_knowledge_mode.session.in_memory_session_storage import (
    InMemorySessionStorage,
)
from intergrax.runtime.drop_in_knowledge_mode.session.session_manager import SessionManager

from intergrax.memory.stores.in_memory_user_profile_store import InMemoryUserProfileStore
from intergrax.memory.user_profile_manager import UserProfileManager
from intergrax.memory.user_profile_memory import UserIdentity, UserPreferences, UserProfile


# --------------------------
# Adapters / Backends
# --------------------------

llm_adapter = LLMAdapterRegistry.create(LLMProvider.OLLAMA)


# --------------------------
# Storage + Managers
# --------------------------

# Sessions (in-memory for notebook tests)
session_storage = InMemorySessionStorage()

# User profile (in-memory for notebook tests)
user_profile_store = InMemoryUserProfileStore()
user_profile_manager = UserProfileManager(store=user_profile_store)

# Session manager (wired with user profile manager)
session_manager = SessionManager(
    storage=session_storage,
    user_profile_manager=user_profile_manager,
    organization_profile_manager=None,
    session_memory_consolidation_service=None,
)


# --------------------------
# Runtime Config (baseline)
# --------------------------

config = RuntimeConfig(
    llm_adapter=llm_adapter,
    enable_user_profile_memory=True,
    enable_org_profile_memory=False,
    enable_rag=False,
    enable_websearch=False,
    tools_mode="off",
)

context = RuntimeContext(
    config=config,
    session_manager=session_manager,
    ingestion_service=None,
    context_builder=None,
    rag_prompt_builder=None,
    websearch_prompt_builder=None,
    history_prompt_builder=None,
)

runtime = RuntimeEngine(context=context)


# --------------------------
# Test Data (stable IDs)
# --------------------------

USER_ID = "user_test_07"
SESSION_ID = "session_test_07"

# A minimal, explicit user profile whose system_instructions should be injected
profile = UserProfile(
    identity=UserIdentity(
        user_id=USER_ID,
        display_name="Artur",
        role="AI/LLM Systems Architect",
        domain_expertise="Intergrax runtime, memory layers, RAG/tool pipelines",
        language="en",
        locale="pl-PL",
        timezone="Europe/Warsaw",
    ),
    preferences=UserPreferences(
        preferred_language="en",
        answer_length="short",
        tone="technical",
        no_emojis_in_code=True,
        no_emojis_in_docs=True,
        prefer_markdown=True,
        prefer_code_blocks=True,
        default_project_context="Intergrax Drop-In Knowledge Runtime",
    ),
    system_instructions=(
        "You are talking to Artur. Use a technical, concise style. "
        "Never use emojis in code blocks or technical documentation. "
        "Default project context: Intergrax Drop-In Knowledge Runtime."
    ),
)

# Persist profile (baseline: instructions already exist, no LLM regeneration here)
await user_profile_manager.save_profile(profile)


[intergraxVectorstoreManager] Initialized provider=chroma, collection=intergrax_docs
[intergraxVectorstoreManager] Existing count: 0


## Step 2 — Run a baseline request and verify instruction injection

In this step we will:

1. Create a `RuntimeRequest` with `user_id` + `session_id`.
2. Call `await runtime.run(request)`.
3. Verify (via `answer.debug_trace["instructions"]`) that the final `system` instructions were built **from user_profile**.
4. Verify that the persisted session history contains **only user/assistant turns** (system instructions are not stored).


In [13]:
from intergrax.runtime.drop_in_knowledge_mode.responses.response_schema import RuntimeAnswer, RuntimeRequest

# --------------------------
# Run a baseline request
# --------------------------

request = RuntimeRequest(
    user_id=USER_ID,
    session_id=SESSION_ID,
    message="Say hello and summarize what you know about my preferences in one sentence.",
    instructions=None,  # important: we want ONLY user_profile instructions in this baseline
)

answer : RuntimeAnswer = await runtime.run(request)

print("=== ANSWER ===")
print(answer.answer)

print("\n=== DEBUG: INSTRUCTIONS SOURCES ===")
instructions_debug = answer.debug_trace.get("instructions", {})
print(instructions_debug)

# Hard assertions for baseline:
# - Instructions must exist
# - They must come from user_profile (not request)
assert instructions_debug.get("has_instructions") is True, "Expected instructions to be injected."
sources = instructions_debug.get("sources", {})
assert sources.get("user_profile") is True, "Expected user_profile instructions source to be True."
assert sources.get("request") is not True, "Expected request instructions source to be False/absent."

# --------------------------
# Verify persisted history does NOT contain system message
# --------------------------

history = await session_manager.get_history(session_id=SESSION_ID)

print("\n=== SESSION HISTORY (PERSISTED) ===")
print(f"History length: {len(history)}")
for i, msg in enumerate(history):
    print(f"[{i}] role={msg.role} content={msg.content[:80]!r}")

assert len(history) >= 2, "Expected at least user + assistant messages in persisted history."
assert history[0].role == "user", "First persisted message should be the user message."
assert all(m.role != "system" for m in history), "System instructions must not be persisted in session history."


=== ANSWER ===
Hello Artur. Based on our interaction history, I understand that you are engaged with the Intergrax Drop-In Knowledge Runtime (DIKR) project and prefer technical communication without emojis in code blocks or documentation.

=== DEBUG: INSTRUCTIONS SOURCES ===
{'has_instructions': True, 'sources': {'request': False, 'user_profile': True, 'organization_profile': False}}

=== SESSION HISTORY (PERSISTED) ===
History length: 2
[0] role=user content='Say hello and summarize what you know about my preferences in one sentence.'
[1] role=assistant content='Hello Artur. Based on our interaction history, I understand that you are engaged'


## Step 3 — Precedence test (request.instructions overrides user profile)

Now we validate the **override rule**:

- If `RuntimeRequest.instructions` is provided, it should take precedence over `UserProfile.system_instructions`.
- `debug_trace["instructions"]["sources"]` should indicate that `request=True`.
- The model output should follow the request-level instruction even if it conflicts with the user profile.

We also keep the same session to confirm that only user/assistant turns are persisted (no system messages).


In [14]:
from intergrax.runtime.drop_in_knowledge_mode.responses.response_schema import RuntimeAnswer, RuntimeRequest

# --------------------------
# Precedence test: request.instructions
# --------------------------

override_instructions = (
    "Reply in TWO bullet points only. "
    "Do not greet. "
    "Do not mention Intergrax."
)

request_override = RuntimeRequest(
    user_id=USER_ID,
    session_id=SESSION_ID,
    message="Summarize my preferences now.",
    instructions=override_instructions,
)

answer_override: RuntimeAnswer = await runtime.run(request_override)

print("=== ANSWER (OVERRIDE) ===")
print(answer_override.answer)

print("\n=== DEBUG: INSTRUCTIONS SOURCES (OVERRIDE) ===")
instructions_debug = answer_override.debug_trace.get("instructions", {})
print(instructions_debug)

# Assertions:
assert instructions_debug.get("has_instructions") is True, "Expected instructions to be injected."
sources = instructions_debug.get("sources", {})
assert sources.get("request") is True, "Expected request instructions source to be True."

# Behavior checks (lightweight, not brittle):
# - should not contain a greeting 'Hello'
# - should not mention 'Intergrax'
assert "Hello" not in answer_override.answer, "Expected no greeting due to request override."
assert "Intergrax" not in answer_override.answer, "Expected no Intergrax mention due to request override."

# --------------------------
# Verify persisted history still has no system messages
# --------------------------

history = await session_manager.get_history(session_id=SESSION_ID)

print("\n=== SESSION HISTORY (PERSISTED, AFTER OVERRIDE) ===")
print(f"History length: {len(history)}")
for i, msg in enumerate(history):
    print(f"[{i}] role={msg.role} content={msg.content}")

assert all(m.role != "system" for m in history), "System instructions must not be persisted in session history."


=== ANSWER (OVERRIDE) ===
• Technical, concise communication
• No use of emojis in code blocks or technical documentation

=== DEBUG: INSTRUCTIONS SOURCES (OVERRIDE) ===
{'has_instructions': True, 'sources': {'request': True, 'user_profile': True, 'organization_profile': False}}

=== SESSION HISTORY (PERSISTED, AFTER OVERRIDE) ===
History length: 4
[0] role=user content=Say hello and summarize what you know about my preferences in one sentence.
[1] role=assistant content=Hello Artur. Based on our interaction history, I understand that you are engaged with the Intergrax Drop-In Knowledge Runtime (DIKR) project and prefer technical communication without emojis in code blocks or documentation.
[2] role=user content=Summarize my preferences now.
[3] role=assistant content=• Technical, concise communication
• No use of emojis in code blocks or technical documentation


## Step 4 — Explicit resolution path and per-session caching

This final step makes the baseline flow explicit and verifiable:

1. Load the persisted `ChatSession`.
2. Call `SessionManager.get_user_profile_instructions_for_session(session)` directly.
3. Verify:
   - the returned string equals `UserProfile.system_instructions` (trimmed),
   - the value is cached on the session (`session.user_profile_instructions`),
   - the refresh flag is cleared (`session.needs_user_instructions_refresh == False`).

This documents the *production-minimal* path used by the runtime:
`UserProfile.system_instructions → (Session cached snapshot) → runtime system instructions`.


In [15]:
from intergrax.runtime.drop_in_knowledge_mode.session.chat_session import ChatSession

# --------------------------
# Load session created by previous runtime calls
# --------------------------

session: ChatSession = await session_manager.get_session(SESSION_ID)
assert session is not None, "Expected the session to exist after previous runtime.run() calls."
assert session.user_id == USER_ID, "Expected session.user_id to match the request user_id."

print("=== SESSION (BEFORE) ===")
print("user_profile_instructions:", repr(session.user_profile_instructions))
print("needs_user_instructions_refresh:", session.needs_user_instructions_refresh)

# --------------------------
# Explicit resolution (SessionManager -> UserProfileManager -> UserProfile.system_instructions)
# --------------------------

resolved = await session_manager.get_user_profile_instructions_for_session(session)
assert isinstance(resolved, str) and resolved.strip(), "Expected a non-empty resolved instructions string."

expected = (profile.system_instructions or "").strip()
assert resolved == expected, "Resolved instructions must equal the stored UserProfile.system_instructions (trimmed)."

# Reload session to verify it was persisted with cached snapshot
session_after: ChatSession = await session_manager.get_session(SESSION_ID)
assert session_after is not None

print("\n=== SESSION (AFTER) ===")
print("resolved:", repr(resolved))
print("cached user_profile_instructions:", repr(session_after.user_profile_instructions))
print("needs_user_instructions_refresh:", session_after.needs_user_instructions_refresh)

assert session_after.user_profile_instructions == resolved, "Expected resolved instructions to be cached on the session."
assert session_after.needs_user_instructions_refresh is False, "Expected refresh flag to be cleared after caching."

print("\nOK: UserProfile.system_instructions → cached per-session snapshot → runtime-ready instructions.")


=== SESSION (BEFORE) ===
user_profile_instructions: 'You are talking to Artur. Use a technical, concise style. Never use emojis in code blocks or technical documentation. Default project context: Intergrax Drop-In Knowledge Runtime.'
needs_user_instructions_refresh: False

=== SESSION (AFTER) ===
resolved: 'You are talking to Artur. Use a technical, concise style. Never use emojis in code blocks or technical documentation. Default project context: Intergrax Drop-In Knowledge Runtime.'
cached user_profile_instructions: 'You are talking to Artur. Use a technical, concise style. Never use emojis in code blocks or technical documentation. Default project context: Intergrax Drop-In Knowledge Runtime.'
needs_user_instructions_refresh: False

OK: UserProfile.system_instructions → cached per-session snapshot → runtime-ready instructions.
